## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps

## Data Read in

In [2]:
df = pd.read_csv("data.csv",dtype='str')

In [3]:
len(df)

67

In [4]:
df = df.dropna(subset='Address')

In [5]:
len(df)

24

## Get Coordinates

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [7]:
df.columns

Index(['File Number', 'Entity Type', 'Entity Name', 'Entity Status',
       'Registered\nEffective Date', 'Manager', 'Address', 'Units',
       'Date Purchased, Price', 'Last Mortgage Amount, Lender',
       'Still Own Y/N?'],
      dtype='object')

In [8]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Address'].apply(geocode)

In [9]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Map Work

In [10]:
df.columns

Index(['File Number', 'Entity Type', 'Entity Name', 'Entity Status',
       'Registered\nEffective Date', 'Manager', 'Address', 'Units',
       'Date Purchased, Price', 'Last Mortgage Amount, Lender',
       'Still Own Y/N?', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [11]:
df.Units = df.Units.str.lower()

I think if we can differentiate color of pins based on who managed the LLC that was landlord, (blue for Gottesman red for Drillman for expample) that'd be good and include info such as unit count, and maybe some of the transactional details when users hover over a pin that'd be good.

In [12]:
df[['Date Purchased', 'Last Price']] = df['Date Purchased, Price'].str.split(',',expand=True)

In [13]:
df = df.drop(columns='Date Purchased, Price')

In [14]:
columns_to_keep = [
    'Manager',
    'Address',
    'Units',
    'Last Mortgage Amount, Lender',
    'Still Own Y/N?',
    'Date Purchased',
    'Last Price',
    'geocoded',
    'lat',
    'lon'
]

map_df = df[columns_to_keep]

In [15]:
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap

# Function to create an HTML popup for each marker
def create_popup(row):
    html = f"""
    <div class="popup-content">
        <strong>Manager:</strong> {row['Manager']}<br>
        <strong>Address:</strong> {row['Address']}<br>
        <strong>Units:</strong> {row['Units']}<br>
        <strong>Last Mortgage Amt / Lender:</strong> {row['Last Mortgage Amount, Lender']}<br>
        <strong>Still Own Y/N?:</strong> {row['Still Own Y/N?']}<br>
        <strong>Date Purchased:</strong> {row['Date Purchased']}<br>
        <strong>Last Price:</strong> {row['Last Price']}
    </div>
    """
    return html

# Define a color mapping for Managers
unique_managers = map_df['Manager'].unique()
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
          'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
          'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 
          'gray', 'black', 'lightgray']
manager_color = {manager: colors[i % len(colors)] for i, manager in enumerate(unique_managers)}

# Initialize the map using the first row's lat and lon values
first_lat = float(map_df['lat'].iloc[0])
first_lon = float(map_df['lon'].iloc[0])
m = folium.Map(location=[first_lat, first_lon], zoom_start=12, scrollWheelZoom=False)

# Add custom Mapbox tile layer (replace with your API key)
folium.TileLayer(
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={map_box_api_key}',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# (Optional) Add custom CSS to style popups
custom_css = """
<style>
    .popup-content {
        min-width: 300px;
        font-size: 14px;
        line-height: 1.4;
        color: #333;
        white-space: normal;
        word-wrap: break-word;
    }
    .leaflet-popup, .leaflet-popup-content-wrapper {
        background-color: #f9f9f9;
        border: 1px solid #bbb;
        border-radius: 5px;
        padding: 8px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.1);
    }
    .leaflet-popup-tip {
        background: #f9f9f9;
    }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

# (Optional) Add a title to the map
title_html = '''
    <h3 style="text-align:center; font-family:Arial, sans-serif; font-size:18px; color:#333; margin-top:10px;">
        <b>Fannie Mae Blacklist</b>
    </h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add additional controls
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# Loop through the DataFrame rows to add markers (without using MarkerCluster)
for i, row in map_df.iterrows():
    lat = float(row['lat'])
    lon = float(row['lon'])
    popup_html = create_popup(row)
    # Set marker color based on the Manager
    color = manager_color.get(row['Manager'], 'blue')
    
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(m)

# Create a legend for manager colors
legend_html = '''
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 150px; 
     background-color: white;
     border:2px solid grey;
     z-index:9999;
     font-size:14px;
     padding: 10px;
     ">
     <strong>Legend</strong><br>
'''
for manager, color in manager_color.items():
    legend_html += f'<i style="background:{color};width:12px;height:12px;display:inline-block;margin-right:5px;"></i>{manager}<br>'
legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map (in Jupyter, this will render the map)
m


In [16]:
m.save('index.html')

In [17]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/FannieMaeBlacklist
